In [1]:
%%sh
apt-get install -y unzip
wget https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
unzip -o bank-additional.zip

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 14 not upgraded.
Need to get 172 kB of archives.
After this operation, 580 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 unzip amd64 6.0-23+deb10u1 [172 kB]
Fetched 172 kB in 0s (1006 kB/s)
Selecting previously unselected package unzip.
(Reading database ... 16492 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-23+deb10u1_amd64.deb ...
Unpacking unzip (6.0-23+deb10u1) ...
Setting up unzip (6.0-23+deb10u1) ...
Processing triggers for mime-support (3.62) ...
Archive:  bank-additional.zip
   creating: bank-additional/
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


debconf: delaying package configuration, since apt-utils is not installed
--2020-06-30 10:36:31--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.252.161
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.252.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 432828 (423K) [application/zip]
Saving to: ‘bank-additional.zip’

     0K .......... .......... .......... .......... .......... 11%  711K 1s
    50K .......... .......... .......... .......... .......... 23%  649K 0s
   100K .......... .......... .......... .......... .......... 35%  643K 0s
   150K .......... .......... .......... .......... .......... 47% 84.3M 0s
   200K .......... .......... .......... .....

In [2]:
import pandas as pd
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
data[:10]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
6,59,admin.,married,professional.course,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
8,24,technician,single,professional.course,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
9,25,services,single,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [10]:
import sagemaker
bucket = 'sagemaker-us-east-1-710299592439'
prefix = 'sagemaker/tutorial-autopilot/input'
desired_s3_uri = f"s3://{bucket}/{prefix}"
print(desired_s3_uri)
sess = sagemaker.Session()

uri = sagemaker.s3.S3Uploader.upload(local_path="./bank-additional/bank-additional-full.csv", desired_s3_uri=f"s3://{bucket}/{prefix}", session=sess)
print(uri)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.
'upload_data' method will be deprecated in favor of 'S3Uploader' class (https://sagemaker.readthedocs.io/en/stable/s3.html#sagemaker.s3.S3Uploader) in SageMaker Python SDK v2.


s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/input
s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/input/bank-additional-full.csv


# Deployment endpoint

In [1]:
import boto3, sys

ep_name = 'tutorial-autopilot-best-model'
sm_rt = boto3.Session().client('runtime.sagemaker')

tn=tp=fn=fp=count=0

with open('bank-additional/bank-additional-full.csv') as f:
    lines = f.readlines()
    for l in lines[1:2000]:   # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[-1]         # Store 'yes'/'no' label
        l = l[:-1]            # Remove label
        l = ','.join(l)       # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))

        if 'yes' in label:
            # Sample is positive
            if 'yes' in response:
                # True positive
                tp=tp+1
            else:
                # False negative
                fn=fn+1
        else:
            # Sample is negative
            if 'no' in response:
                # True negative
                tn=tn+1
            else:
                # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
recall    = tn/(tn+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))

ValidationError: An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint tutorial-autopilot-best-model of account 710299592439 not found.

# Cleanup


In [6]:
bucket = 'sagemaker-us-east-1-710299592439'
desired_s3_uri = f"s3://{bucket}/sagemaker/tutorial-autopilot/"
sess.delete_endpoint(endpoint_name=ep_name)

NameError: name 'sess' is not defined

In [8]:
%%sh
aws s3 rm --recursive s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/

delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/input/bank-additional-full.csv
delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/output/tutorial-autopilot/data-processor-models/tutorial-a-dpp5-1-11e01672009c4528a34218cbc819077941a712a35ab14/output/model.tar.gz
delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/output/tutorial-autopilot/data-processor-models/tutorial-a-dpp1-1-b3a73605f7314a76be641fd8667880788e4ffb513a454/output/model.tar.gz
delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/output/tutorial-autopilot/data-processor-models/tutorial-a-dpp0-1-27162f4b0a4e4e1cb3beacf1d396d482b2c6b9faac044/output/model.tar.gz
delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-autopilot/output/tutorial-autopilot/data-processor-models/tutorial-a-dpp2-1-4023cc2c8b7a4d909c460a2befba09197c0ec17ffb1f4/output/model.tar.gz
delete: s3://sagemaker-us-east-1-710299592439/sagemaker/tutorial-a